Fist we need to import the vision libs

In [1]:
%pip install --upgrade google-cloud-vision
%pip install pillow
%pip install --upgrade Pillow
%pip install hexdump

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


The credential we use to access the service then need to be associated with the current session


In [6]:
import os

# Set the environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'G:\My Drive\credential\imagelable-379418-7b90ef4ee1f0.json'
work_dir = 'C:/temp/2023Mar/Output/web'

Functions used in the tools

This code reads an image file and sends it for analysis


In [19]:
import os
import io
from PIL import Image

def showImageIptcDataDirectory(directory):
  # Iterate over each file in the directory
  for file_name in os.listdir(directory):
    # Check if the file is an image file
    if file_name.endswith('.jpg') or file_name.endswith('.jpeg') or file_name.endswith('.png'):
      showImageIptcMeta(directory, file_name)

def showImageIptcMeta(directory, file_name):
  outputDelimiter = '#'
  with Image.open(os.path.join(directory, file_name)) as img:
    # Get the IPTC metadata
    iptc = img.info.get('photoshop')
    # there is actually just one key 1028 - this contains the keywords and
    # other meta data
    iptcInfo = iptc[1028]
    # parse the existing meta date
    results = parse_iptc_data(iptcInfo)
    title = results['Object Attribute']
    description = ''
    if 'Caption/Abstract Writer' in results:
        description = results['Caption/Abstract Writer']

    print('title:' + title)
    print('description:' + description)
    keywords = results['Keywords']
    processed_keywords = [''.join([word.capitalize() for word in keyword.split()]) for keyword in keywords]

    
    keyword_string = ' '.join([outputDelimiter + kw for kw in processed_keywords])
    print(keyword_string)
    print(file_name)
    print("<--------------------------------------------------------------->")

def parse_iptc_data(iptc_data):
    """#
    Parse IPTC binary data and return a dictionary of IPTC properties.

    :param iptc_data: A byte array containing IPTC binary data.
    :returns: A dictionary containing IPTC properties.
    """
    #hexdump.hexdump(iptc_data, 'print')
    results = {}
    tag_start = b'\x1c\x02'
    pos = 0

    while True:
        start = iptc_data.find(tag_start, pos)
        if start < 0:
            break

        typeMajor = int.from_bytes(iptc_data[start+1:start+2], byteorder='big')
        typeMinor = int.from_bytes(iptc_data[start+2:start+3], byteorder='big')
        tag_type = str(typeMajor) + ":" + str(typeMinor)
        tag_len = int.from_bytes(iptc_data[start+3:start+5], byteorder='big')
        tag_end = start + 5 + tag_len 
        
        tag_value = iptc_data[start+5:tag_end].decode('utf-8')
        # print(start, tag_type, tag_len, tag_end, tag_value)
        tag_key = IPTC_TAG_TYPES.get(str(tag_type), 'Unknown')
        if tag_key in results:
            if not isinstance(results[tag_key], list):
                results[tag_key] = [results[tag_key]]
            results[tag_key].append(tag_value)
        else:
            results[tag_key] = tag_value
        pos = tag_end
    return results

from PIL import Image



def merge_keywords(iptc_data, new_keywords): 
    existing_keywords = iptc_data.get('Keywords', [])
    #print(existing_keywords)
    merged_keywords = list(set(existing_keywords + new_keywords))
    #print(merged_keywords)
    iptc_data['Keywords'] = merged_keywords
    #print(iptc_data)
    return iptc_data

def encode_iptc_data(iptc_data):
    encoded_data = b''
    
    # Create the IPTC header
    record_version = b'\x1c\x02\x00\x00\x02\x00\x04'
    application_record = b'\x1c\x02\x05\x00\x10Adobe Photoshop 22.0'
    date_created = iptc_data.get('Digital Creation Date/Time', '').encode('utf-8')
    time_created = iptc_data.get('Time Created', '').encode('utf-8')
    digital_creation_date_time = b'\x1c\x02\x19\x00' + date_created + b'\x1c\x02\x1a\x00' + time_created
    header = record_version + application_record + digital_creation_date_time
    
    # Add the IPTC tags
    for tag_key, tag_value in iptc_data.items():
        tag_type = IPTC_TAG_TYPES_INV.get(tag_key, None)
        if tag_type is not None:
            if isinstance(tag_value, list):
                for value in tag_value:
                    encoded_data += encode_iptc_tag(tag_type, value)
            else:
                encoded_data += encode_iptc_tag(tag_type, tag_value)
    
    # Concatenate the header and IPTC data
    encoded_data = header + encoded_data
    
    return encoded_data

def encode_iptc_data2(iptc_data):
    encoded_data = b''
    for tag_key, tag_value in iptc_data.items():
        tag_type = IPTC_TAG_TYPES_INV.get(tag_key, None)
        #print(str(tag_key), "---", tag_type, "---", type(tag_value), "---", tag_value)
        if tag_type is not None:
            if isinstance(tag_value, list):
                for value in tag_value:
                    encoded_data += encode_iptc_tag(tag_type, value)
                    #print(encoded_data)
            else:
                encoded_data += encode_iptc_tag(tag_type, tag_value)
                #print(encoded_data)
    return encoded_data

def encode_iptc_tag(tag_type, tag_value):
    tag_major, tag_minor = tag_type.split(':')
    #print(tag_major, tag_minor)
    tag_major = int(tag_major)
    tag_minor = int(tag_minor)
    #print(tag_major, tag_minor)

    tag_value_bytes = tag_value.encode('utf-8')
    tag_len = len(tag_value_bytes)
    if tag_len > 65535:
        raise ValueError('Tag length too long')

    encoded_tag = b'\x1c' + tag_major.to_bytes(1, byteorder='big') + \
        tag_minor.to_bytes(1, byteorder='big') + \
        tag_len.to_bytes(2, byteorder='big') + tag_value_bytes 

    #print (encoded_tag)
    return encoded_tag


from google.cloud import vision

def anotate_image(file_name):
    client = vision.ImageAnnotatorClient()
    with io.open(file_name, 'rb') as image_file:
        content = image_file.read() 
        image = vision.Image(content=content)

        # Perform label detection on the image file
        response = client.label_detection(image=image)
        labels = response.label_annotations
        return labels


def anotate_folder(work_dir):
    # Set the minimum topicality score
    min_topicality = 0.55
    # outputDelimiter = '#'
    outputDelimiter = ''

    # Iterate over each file in the directory
    for file_name in os.listdir(work_dir):
        # Check if the file is an image file
        if file_name.endswith('.jpg') or file_name.endswith('.jpeg') or file_name.endswith('.png'):
            full_path_name = os.path.join(work_dir, file_name)
            labels = anotate_image(full_path_name)
            for label in labels:
                print(label.description, label.topicality)
            # Format the labels as a comma-separated list with hash tags, remove hyphens and spaces, and filter based on topicality score
            label_list = [label.description.replace('-', '').replace(' ', '').lower() for label in labels if label.topicality >= min_topicality]
            label_string = ','.join([outputDelimiter + label for label in label_list])

            # Print the formatted labels for each image file
            print(file_name, label_string)


IPTC_TAG_TYPES = {
    "2:0": "Record Version",
    "2:3": "Object Type",
    "2:5": "Object Attribute",
    "2:7": "Object Name",
    "2:10": "Edit Status",
    "2:12": "Editorial Update",
    "2:15": "Urgency",
    "2:20": "Keyword",
    "2:22": "Category",
    "2:25": "Keywords",
    "2:26": "Location",
    "2:27": "City",
    "2:30": "Caption/Abstract",
    "2:40": "Instructions",
    "2:55": "Date Created",
    "2:60": "Time Created",
    "2:62": "Digital Creation Date/Time",
    "2:63": "Originating Program",
    "2:80": "Byline",
    "2:85": "Byline Title",
    "2:90": "City",
    "2:92": "Sublocation",
    "2:95": "State/Province",
    "2:100": "Country/Primary Location Name",
    "2:101": "Country/Primary Location Code",
    "2:103": "Original Transmission Reference",
    "2:105": "Headline",
    "2:110": "Credit",
    "2:115": "Source",
    "2:116": "Copyright Notice",
    "2:118": "Contact",
    "2:120": "Caption/Abstract Writer",
    "2:122": "Rasterized Caption",
    "2:130": "Content Location Code",
    "2:131": "Content Location Name",
    "2:135": "ICC Profile",
    "2:150": "Writer/Editor",
    "2:151": "Image Type",
    "2:184": "Job ID",
    "2:185": "Master Document ID",
    "2:186": "Short Document ID",
    "2:187": "Unique Document ID",
    "2:188": "Owner ID",
    "2:200": "Object Preview File Format",
    "2:201": "Object Preview File Format Version",
    "2:202": "Object Preview Data"
}

IPTC_TAG_TYPES_INV = {v: k for k, v in IPTC_TAG_TYPES.items()}




In [24]:
# directory = 'G:\My Drive\images'
#showImageIptcDataDirectory(work_dir)
anotate_folder(work_dir)

Screenshot 2023-03-22 122739.png cloud,sky,water,atmosphere,blackandwhite,flashphotography,sunlight,style,rectangle,fence


Attempts at IPTC processing

In [ ]:
from PIL import Image
# Open the JPEG file
img = Image.open('/content/drive/MyDrive/images/R0002558.jpg')
img.save('/content/drive/MyDrive/images/imageBak.jpg')

# Get the IPTC metadata
iptc = img.info.get('photoshop')

#get the info dict
myinfo = img.info

# there is actually just one key 1028 - this contains the keywords and
# other meta data
iptcInfo = iptc[1028]
# parse the existing meta date
print("<---------------------- originat meta data below ----------------------------------------->")
results = parse_iptc_data(iptcInfo)
kk = results['Keywords']
print(results)

#merge some new keywords
newnames = ["k1", "wales", "k99"]
newInfo = merge_keywords(results, newnames)
encoded_info = encode_iptc_data2(newInfo)



# add iptc header annotation
header = b'\x1c\x01Z\x00\x03\x1b%G'
header += encoded_info
encode_info = header
#print(header)

# parse the new version of encoded info for testing
print("<----------------------- modified meta data below ---------------------------------------->")
results2 = parse_iptc_data(header)
print(results2)

# attempt to udpate files

iptc[1028] = encode_info
img.info['photoshop'] = iptc
img.save('/content/drive/MyDrive/images/imageZAA99.jpg', format='JPEG', iptc=iptc)


# Open the JPEG file
#img = Image.open('/content/drive/MyDrive/images/imageZAA.jpg')

# Get the IPTC metadata
iptc = img.info.get('photoshop')

#get the info dict
myinfo = img.info

# there is actually just one key 1028
iptcInfo = iptc[1028]
#print("<--------------------------------------------------------------->")
results = parse_iptc_data(iptcInfo)
print(results)

print("<--------------------------------------------------------------->")


In [ ]:
!pip install piexif
import piexif
from PIL import Image

# Open the source JPEG file
src_img = Image.open('/content/drive/MyDrive/images/R0002605_1.jpg')
src_exif = piexif.load(src_img.info['exif'])

# Open the destination JPEG file
dst_img = Image.open('/content/drive/MyDrive/images/imageAA1.jpg')
dst_exif = piexif.load(dst_img.info['exif'])

# Copy the IPTC metadata from the source to the destination
if piexif.ImageIFD.XPKeywords in src_exif['0th']:
    dst_exif['0th'][piexif.ImageIFD.XPKeywords] = src_exif['0th'][piexif.ImageIFD.XPKeywords]
else:
    print("XPKeywords not found in source image metadata.")


# Save the destination JPEG file with the updated metadata
dst_exif_bytes = piexif.dump(dst_exif)
dst_img.save('/content/drive/MyDrive/images/imageAA1.jpg', format='JPEG', exif=dst_exif_bytes)


In [ ]:
from PIL import Image
from PIL.IptcImagePlugin import IptcImageFile

# Open an image file
img = Image.open("/content/drive/MyDrive/images/R0002590clr.jpg")

# Access the IPTC data
iptczz = IptcImageFile(img)

# Display the IPTC data
print(iptc.iptc_data)